In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

# Importing Data

In [2]:
partd_2018 = pd.read_csv('/Volumes/ML_projects/Medicare_Fraud_Datasets/PARTD/PartD_Prescriber_PUF_NPI_Drug_18.txt', sep='\t')
partb_2018 = pd.read_csv('/Volumes/ML_projects/Medicare_Fraud_Datasets/PARTB/Medicare_Provider_Util_Payment_PUF_CY2018/Medicare_Provider_Util_Payment_PUF_CY2018.txt', sep='\t')
dmepos_2018 = pd.read_csv('/Volumes/ML_projects/Medicare_Fraud_Datasets/DMEPOS/Medicare_Referring_Provider_DMEPOS_PUF_CY2018/Medicare_Referring_Provider_DMEPOS_PUF_CY2018.csv')
partd_2017 = pd.read_csv('/Volumes/ML_projects/Medicare_Fraud_Datasets/PARTD/PartD_Prescriber_PUF_NPI_DRUG_17/PartD_Prescriber_PUF_NPI_Drug_17.txt', sep='\t')
partb_2017 = pd.read_csv('/Volumes/ML_projects/Medicare_Fraud_Datasets/PARTB/Medicare_Provider_Util_Payment_PUF_CY2017/Medicare_Provider_Util_Payment_PUF_CY2017.txt', sep='\t')
dmepos_2017 = pd.read_csv('/Volumes/ML_projects/Medicare_Fraud_Datasets/DMEPOS/medicare_referring_provider_dmepos_puf_cy2017/Medicare_Referring_Provider_DMEPOS_PUF_CY2017.tab', sep='\t')
partd_2016 = pd.read_csv('/Volumes/ML_projects/Medicare_Fraud_Datasets/PARTD/PartD_Prescriber_PUF_NPI_DRUG_16/PartD_Prescriber_PUF_NPI_Drug_16.txt', sep='\t')
partb_2016 = pd.read_csv('/Volumes/ML_projects/Medicare_Fraud_Datasets/PARTB/Medicare-Physician-and-Other-Supplier-PUF/Medicare_Provider_Util_Payment_PUF_CY2016.txt', sep='\t')
dmepos_2016 = pd.read_csv('/Volumes/ML_projects/Medicare_Fraud_Datasets/DMEPOS/medicare_referring_provider_dmepos_puf_cy2016/Medicare_Referring_Provider_DMEPOS_PUF_CY2016.tab', sep='\t')

In [3]:
leie = pd.read_csv('/Volumes/ML_projects/Medicare_Fraud_Datasets/LEIE.csv')

# Selecting features and targets.

Filtering out specific exclusion codes for more severe offenses. These will be used to create `FRAUD` and `NOT FRAUD` targets.

In [4]:
#mandatory minimum penalty based on offense category

minimum_exclusion_periods = { 
    '1128a1':5, #Conviction of program-related crimes. Minimum Period: 5 years
    '1128a2':5, #Conviction relating to patient abuse or neglect. Minimum Period: 5 years
    '1128a3':5, #Felony conviction relating to health care fraud. Minimum Period: 5 years
    '1128b4':np.nan, #License revocation, suspension, or surrender. Minimum Period: Period imposed by the state licensing authority.    
    '1128b7':np.nan, #Fraud, kickbacks, and other prohibited activities. Minimum Period: None
    '1128c3gi':10, #Conviction of second mandatory exclusion offense. Minimum Period: 10 years
    '1128c3gii':100 #Conviction of third or more mandatory exclusion offenses. Permanent Exclusion
}

In [5]:
exclusion_codes = ['1128a1', '1128a2', '1128a3', '1128b4', '1128b7', '1128c3gi', '1128c3gii']
filtered_leie = leie[leie.EXCLTYPE.isin(exclusion_codes)]
filtered_leie.shape

(65309, 18)

Converting the dates into a datetime.

In [6]:
filtered_leie['EXCLDATE'] = pd.to_datetime(filtered_leie.EXCLDATE, format='%Y%m%d') #most important value
filtered_leie['WAIVERDATE'] = pd.to_datetime(filtered_leie.WAIVERDATE, format='%Y%m%d', errors='coerce')
filtered_leie['REINDATE'] = pd.to_datetime(filtered_leie.REINDATE, format='%Y%m%d', errors='coerce')

As we can see, there are only 11 dates for the `WAIVERDATE` column and only 1 for the `REINDATE` column, so to infer the exclusion end dates we can add the `minimum_exclusion_periods` value to the exclusion date (`EXCLDATE`) column.

In [7]:
print("Unique `WAIVERDATE` values: ", len(filtered_leie['WAIVERDATE'].unique()))
print("Unique `REINDATE` values: ", len(filtered_leie['REINDATE'].unique()))

Unique `WAIVERDATE` values:  11
Unique `REINDATE` values:  1


In [8]:
#mapping the penalty to the minimum exclusions period, placing minimum exclusions on their own col

filtered_leie['MIN_EXCLUSION_PERIOD'] = filtered_leie['EXCLTYPE'].map(minimum_exclusion_periods)

Now we have to add the minimum exclusion periods to the original exclusion dates. This will give us an estimate of the exclusion end date. 

In [9]:
filtered_leie['END_EXCLDATE'] = filtered_leie.EXCLDATE.dt.year + filtered_leie.MIN_EXCLUSION_PERIOD

The `calculate_exclusion_end` function takes in the beginning exclusion date, then if the month is greater that `6` , adds a year to the end year. This is because the provider would have been fraudulant for most of the year.

In [10]:
def calculate_exclusion_end(exclusion_date, end_year):
    month = exclusion_date.month
    

    if month > 6:
        return end_year + 1
    else:
        return end_year
        

In [11]:
#applying the function to both columns

filtered_leie['END_EXCLDATE'] = filtered_leie[["EXCLDATE","END_EXCLDATE"]].apply(lambda x: calculate_exclusion_end(*x), axis=1)

In [12]:
filtered_leie

,LASTNAME,FIRSTNAME,MIDNAME,BUSNAME,GENERAL,SPECIALTY,UPIN,NPI,DOB,ADDRESS,CITY,STATE,ZIP,EXCLTYPE,EXCLDATE,REINDATE,WAIVERDATE,WVRSTATE,MIN_EXCLUSION_PERIOD,END_EXCLDATE
0,NaN,NaN,,"#1 MARKETING SERVICE, INC",OTHER BUSINESS,SOBER HOME,NaN,0,NaN,239 BRIGHTON BEACH AVENUE,BROOKLYN,NY,11235,1128a1,2020-03-19,NaT,NaT,NaN,5.0,2025.0
1,NaN,NaN,,14 LAWRENCE AVE PHARMACY,PHARMACY,NaN,NaN,0,NaN,14 LAWRENCE AVENUE,SMITHTOWN,NY,11787,1128a1,1988-08-30,NaT,NaT,NaN,5.0,1994.0
2,NaN,NaN,,143 MEDICAL EQUIPMENT CO,DME COMPANY,DME - OXYGEN,NaN,0,NaN,701 NW 36 AVENUE,MIAMI,FL,33125,1128b7,1997-06-20,NaT,NaT,NaN,NaN,NaN
3,NaN,NaN,,184TH STREET PHARMACY CORP,OTHER BUSINESS,PHARMACY,NaN,1922348218,NaN,69 E 184TH ST,BRONX,NY,10468,1128a1,2018-04-19,NaT,NaT,NaN,5.0,2023.0
5,NaN,NaN,,"1ST COMMUNITY HEALTH CTR, LTD",CLINIC,NaN,NaN,0,NaN,3138 W CERMAK ROAD,CHICAGO,IL,60623,1128a1,1994-05-24,NaT,NaT,NaN,5.0,1999.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74380,ZWOLINSKI,AMBER,DAWN,NaN,IND- LIC HC SERV PRO,NURSE/NURSES AIDE,NaN,0,19870529.0,1238 PULASKI AVENUE,COAL TOWNSHIP,PA,17866,1128b4,2011-12-20,NaT,NaT,NaN,NaN,NaN
74381,ZWOLINSKI,AMBER,DAWN,NaN,IND- LIC HC SERV PRO,NURSE/NURSES AIDE,NaN,0,19870529.0,1238 PULASKI AVENUE,COAL TOWNSHIP,PA,17866,1128a1,2016-05-19,NaT,NaT,NaN,5.0,2021.0
74382,ZYDERVELD,MARY,ANN,NaN,PHARMACY,TECHNICIAN,NaN,0,19520825.0,1358 MORNING GLORY TURN,RUCKERSVILLE,VA,22962,1128b4,2009-02-19,NaT,NaT,NaN,NaN,NaN
74383,ZYLSTRA,CHRISTEENA,LYNN,NaN,NURSING PROFESSION,NURSE/NURSES AIDE,NaN,0,19670724.0,1816 NE 49TH STREET,KANSAS CITY,MO,64118,1128b4,2010-03-18,NaT,NaT,NaN,NaN,NaN


Filtering out features from the three datasets.

In [13]:
partb_feats = ['npi', 'hcpcs_code', 'hcpcs_description', 'hcpcs_drug_indicator', 'provider_type', 'nppes_provider_gender', 'line_srvc_cnt', 'bene_unique_cnt', 'bene_day_srvc_cnt', 'average_submitted_chrg_amt', 'average_Medicare_payment_amt']
partd_feats = ['npi', 'specialty_description', 'bene_count', 'total_claim_count', 'total_30_day_fill_count', 'total_day_supply', 'total_drug_cost']
dmepos_feats = ['REFERRING_NPI', 'REFERRING_PROVIDER_TYPE', 'REFERRING_PROVIDER_GENDER', 'NUMBER_OF_SUPPLIERS', 'NUMBER_OF_SUPPLIER_BENEFICIARIES', 'NUMBER_OF_SUPPLIER_CLAIMS', 'NUMBER_OF_SUPPLIER_SERVICES', 'AVG_SUPPLIER_SUBMITTED_CHARGE', 'AVG_SUPPLIER_MEDICARE_PMT_AMT']

partb_features_2018 = partb_2018[partb_feats]
partd_features_2018 = partd_2018[partd_feats]
dmepos_features_2018 = dmepos_2018[dmepos_feats]

#Add a year column so we know where to add fraud labels. Activity before exclusion dates will be considered fraudulent. 
partb_features_2018['DATA_YEAR'] = 2018
partd_features_2018['DATA_YEAR'] = 2018
dmepos_features_2018['DATA_YEAR'] = 2018

partb_features_2017 = partb_2017[partb_feats]
partd_features_2017 = partd_2017[partd_feats]
dmepos_features_2017 = dmepos_2017[dmepos_feats]

partb_features_2017['DATA_YEAR'] = 2017
partd_features_2017['DATA_YEAR'] = 2017
dmepos_features_2017['DATA_YEAR'] = 2017

partb_features_2016 = partb_2016[[x.upper() for x in partb_feats]]
partd_features_2016 = partd_2016[partd_feats]
dmepos_features_2016 = dmepos_2016[dmepos_feats]

#make 2016 headers lowercase like the other years. 
partb_features_2016.columns = [x.lower() for x in partb_features_2016.columns]

partb_features_2016['DATA_YEAR'] = 2016
partd_features_2016['DATA_YEAR'] = 2016
dmepos_features_2016['DATA_YEAR'] = 2016

In [14]:
partb_features_2016

,npi,hcpcs_code,hcpcs_description,hcpcs_drug_indicator,provider_type,nppes_provider_gender,line_srvc_cnt,bene_unique_cnt,bene_day_srvc_cnt,average_submitted_chrg_amt,average_medicare_payment_amt,DATA_YEAR
0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016
1,1003000126,99217,Hospital observation care discharge,N,Internal Medicine,M,57.0,55.0,57.0,325.157895,54.474386,2016
2,1003000126,99219,Hospital observation care typically 50 minutes,N,Internal Medicine,M,38.0,38.0,38.0,614.000000,105.850000,2016
3,1003000126,99220,Hospital observation care typically 70 minutes...,N,Internal Medicine,M,23.0,23.0,23.0,769.347826,146.116522,2016
4,1003000126,99221,"Initial hospital inpatient care, typically 30 ...",N,Internal Medicine,M,20.0,20.0,20.0,460.000000,78.990000,2016
...,...,...,...,...,...,...,...,...,...,...,...,...
9714892,1992999874,99223,"Initial hospital inpatient care, typically 70 ...",N,Internal Medicine,F,138.0,136.0,138.0,496.000000,155.637826,2016
9714893,1992999874,99232,"Subsequent hospital inpatient care, typically ...",N,Internal Medicine,F,315.0,167.0,315.0,176.000000,56.270000,2016
9714894,1992999874,99233,"Subsequent hospital inpatient care, typically ...",N,Internal Medicine,F,323.0,175.0,323.0,254.000000,80.918700,2016
9714895,1992999874,99239,"Hospital discharge day management, more than 3...",N,Internal Medicine,F,160.0,155.0,160.0,262.000000,83.393938,2016


# Concatenate all the yearly data.

In [15]:
# make NPI lowercase so we can merge on it later. 

filtered_leie.rename({'NPI':'npi'}, axis=1, inplace=True)
dmepos_features_2018.rename({'REFERRING_NPI':'npi'}, axis=1, inplace=True)
dmepos_features_2017.rename({'REFERRING_NPI':'npi'}, axis=1, inplace=True)
dmepos_features_2016.rename({'REFERRING_NPI':'npi'}, axis=1, inplace=True)

In [16]:
partb_features = pd.concat([partb_features_2018, partb_features_2017, partb_features_2016])
partd_features = pd.concat([partd_features_2018, partd_features_2017, partd_features_2016])
dmepos_features = pd.concat([dmepos_features_2018, dmepos_features_2017, dmepos_features_2016])

# Part B dataset processing.

Filter out HCPCS codes referring to prescriptions, the `line_srvc_cnt` feature for these entries refer to weight/volume of a drug and not procedure counts. Including both in the same dataset would cause conflicts. 

In [17]:
partb_features = partb_features[partb_features.hcpcs_drug_indicator.eq('N')]

In [18]:
partb_features.head(5).iloc[1]

DATA_YEAR                                                                  2018
average_Medicare_payment_amt                                            76.7953
average_medicare_payment_amt                                                NaN
average_submitted_chrg_amt                                              476.947
bene_day_srvc_cnt                                                            19
bene_unique_cnt                                                              19
hcpcs_code                                                                99218
hcpcs_description               Hospital observation care, typically 30 minutes
hcpcs_drug_indicator                                                          N
line_srvc_cnt                                                                19
npi                                                                  1003000126
nppes_provider_gender                                                         M
provider_type                           

In [19]:
partb_features = partb_features.groupby(['npi','provider_type', 'nppes_provider_gender', 'DATA_YEAR']).agg({'line_srvc_cnt':['sum', 'mean', 'median', np.std, 'min', 'max'],
                                                                                     'bene_unique_cnt':['sum', 'mean', 'median', np.std, 'min', 'max'],
                                                                                     'bene_day_srvc_cnt':['sum', 'mean', 'median', np.std, 'min', 'max'],
                                                                                     'average_submitted_chrg_amt':['sum', 'mean', 'median', np.std, 'min', 'max'],
                                                                                     'average_medicare_payment_amt': ['sum', 'mean', 'median', np.std, 'min', 'max']})

Flatten the multiindex to make it easier to work with.

In [20]:
partb_features.columns = ['_'.join(col) for col in partb_features.columns.values]

Physicians who only performed and operation or perscribed one type of drug or procedure show as `null` values, we can replace these values with 0 because there is no variability. 

In [21]:
partb_features.isna().sum()

line_srvc_cnt_sum                            0
line_srvc_cnt_mean                           0
line_srvc_cnt_median                         0
line_srvc_cnt_std                       317799
line_srvc_cnt_min                            0
line_srvc_cnt_max                            0
bene_unique_cnt_sum                          0
bene_unique_cnt_mean                         0
bene_unique_cnt_median                       0
bene_unique_cnt_std                     317799
bene_unique_cnt_min                          0
bene_unique_cnt_max                          0
bene_day_srvc_cnt_sum                        0
bene_day_srvc_cnt_mean                       0
bene_day_srvc_cnt_median                     0
bene_day_srvc_cnt_std                   317799
bene_day_srvc_cnt_min                        0
bene_day_srvc_cnt_max                        0
average_submitted_chrg_amt_sum               0
average_submitted_chrg_amt_mean              0
average_submitted_chrg_amt_median            0
average_submi

In [22]:
#replacing NA with 0

partb_features.fillna(0, inplace=True)

In [23]:
partb_features = partb_features.reset_index(level=['npi', 'provider_type', 'nppes_provider_gender', 'DATA_YEAR'])

# Part D dataset processing.

In [24]:
partd_features.columns

Index(['npi', 'specialty_description', 'bene_count', 'total_claim_count',
       'total_30_day_fill_count', 'total_day_supply', 'total_drug_cost',
       'DATA_YEAR'],
      dtype='object')

Beneficiary counts were supressed to 0 if they were below 11, so we replace 0's with 5's as per CMS documentation.

In [25]:
partd_features.bene_count = partd_features.bene_count.replace(0,5)

Aggregate over numerical columns.

In [26]:
partd_features = partd_features.groupby(['npi','specialty_description', 'DATA_YEAR']).agg({'bene_count':['sum', 'mean', 'median', np.std, 'min', 'max'],
                                                     'total_claim_count':['sum', 'mean', 'median', np.std, 'min', 'max'],
                                                     'total_30_day_fill_count':['sum', 'mean', 'median', np.std, 'min', 'max'],
                                                     'total_day_supply':['sum', 'mean', 'median', np.std, 'min', 'max'],
                                                     'total_drug_cost': ['sum', 'mean', 'median', np.std, 'min', 'max']})

Flattening the multiindex!

In [27]:
partd_features.columns = ['_'.join(col) for col in partd_features.columns.values]

In [28]:
partd_features = partd_features.reset_index(level=['specialty_description', 'npi', 'DATA_YEAR'])

In [29]:
partd_features

,npi,specialty_description,DATA_YEAR,bene_count_sum,bene_count_mean,bene_count_median,bene_count_std,bene_count_min,bene_count_max,total_claim_count_sum,...,total_day_supply_median,total_day_supply_std,total_day_supply_min,total_day_supply_max,total_drug_cost_sum,total_drug_cost_mean,total_drug_cost_median,total_drug_cost_std,total_drug_cost_min,total_drug_cost_max
0,1003000126,Internal Medicine,2016,142.0,15.777778,14.0,4.763869,11.0,26.0,310,...,420.0,348.403051,96,1050,20655.60,1377.040000,169.480,3335.122401,45.76,12110.20
1,1003000126,Internal Medicine,2017,232.0,19.333333,16.5,8.978088,11.0,35.0,385,...,385.0,421.564502,65,1530,12374.60,773.412500,183.295,2284.669976,43.88,9326.11
2,1003000126,Internal Medicine,2018,143.0,15.888889,14.0,5.754226,11.0,28.0,252,...,391.0,282.534915,64,885,8174.86,583.918571,131.855,1701.559376,39.02,6494.22
3,1003000142,Anesthesiology,2016,351.0,27.000000,18.0,19.680786,11.0,69.0,1633,...,1007.5,2795.322855,295,10092,120865.38,5493.880909,1498.140,11242.693578,31.88,49748.43
4,1003000142,Anesthesiology,2017,433.0,30.928571,19.0,26.137107,11.0,90.0,1846,...,1309.0,3130.959222,289,11835,113868.77,5175.853182,2217.470,11001.480212,161.78,52305.09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2764425,1992999833,Psychoanalyst,2018,0.0,NaN,NaN,NaN,NaN,NaN,11,...,290.0,NaN,290,290,16057.29,16057.290000,16057.290,NaN,16057.29,16057.29
2764426,1992999866,Dentist,2016,0.0,NaN,NaN,NaN,NaN,NaN,12,...,84.0,NaN,84,84,218.42,218.420000,218.420,NaN,218.42,218.42
2764427,1992999874,Internal Medicine,2016,31.0,15.500000,15.5,3.535534,13.0,18.0,31,...,139.0,59.396970,97,181,325.50,162.750000,162.750,116.460487,80.40,245.10
2764428,1992999882,Physician Assistant,2016,83.0,13.833333,12.5,3.544949,11.0,20.0,216,...,443.0,205.053042,316,1037,1961.22,163.435000,170.265,95.267921,39.24,360.38


# DMEPOS dataset processing.

In [30]:
dmepos_features.columns

Index(['npi', 'REFERRING_PROVIDER_TYPE', 'REFERRING_PROVIDER_GENDER',
       'NUMBER_OF_SUPPLIERS', 'NUMBER_OF_SUPPLIER_BENEFICIARIES',
       'NUMBER_OF_SUPPLIER_CLAIMS', 'NUMBER_OF_SUPPLIER_SERVICES',
       'AVG_SUPPLIER_SUBMITTED_CHARGE', 'AVG_SUPPLIER_MEDICARE_PMT_AMT',
       'DATA_YEAR'],
      dtype='object')

Beneficiary counts were supressed to 0 if they were below 11, so we replace 0's with 5's as per CMS documentation.

In [31]:
dmepos_features.NUMBER_OF_SUPPLIER_BENEFICIARIES = dmepos_features.NUMBER_OF_SUPPLIER_BENEFICIARIES.replace(0,5)

In [32]:
dmepos_features = dmepos_features.groupby(['npi','REFERRING_PROVIDER_TYPE', 
                         'REFERRING_PROVIDER_GENDER', 'DATA_YEAR']).agg({'NUMBER_OF_SUPPLIERS':['sum', 'mean', 'median', np.std, 'min', 'max'],
                                                                 'NUMBER_OF_SUPPLIER_BENEFICIARIES':['sum', 'mean', 'median', np.std, 'min', 'max'],
                                                                 'NUMBER_OF_SUPPLIER_CLAIMS':['sum', 'mean', 'median', np.std, 'min', 'max'],
                                                                 'NUMBER_OF_SUPPLIER_SERVICES':['sum', 'mean', 'median', np.std, 'min', 'max'],
                                                                 'AVG_SUPPLIER_MEDICARE_PMT_AMT':['sum', 'mean', 'median', np.std, 'min', 'max'],
                                                                 'AVG_SUPPLIER_SUBMITTED_CHARGE': ['sum', 'mean', 'median', np.std, 'min', 'max']})

Flattening the multiindex!

In [33]:
dmepos_features.columns = ['_'.join(col) for col in dmepos_features.columns.values]

In [34]:
dmepos_features = dmepos_features.reset_index(level=['REFERRING_PROVIDER_TYPE', 'npi', 'DATA_YEAR', 'REFERRING_PROVIDER_GENDER'])

# Merge all the combined yearly data on with `LEIE` on NPI

In [35]:
partb = pd.merge(filtered_leie, partb_features, on='npi', how='outer')
partd = pd.merge(filtered_leie, partd_features, on='npi', how='outer')
dmepos = pd.merge(filtered_leie, dmepos_features, on='npi', how='outer')

In [36]:
partb = partb[['npi', 'provider_type', 'DATA_YEAR', 'nppes_provider_gender', 'line_srvc_cnt_sum',
       'line_srvc_cnt_mean', 'line_srvc_cnt_median', 'line_srvc_cnt_std',
       'line_srvc_cnt_min', 'line_srvc_cnt_max', 'bene_unique_cnt_sum',
       'bene_unique_cnt_mean', 'bene_unique_cnt_median', 'bene_unique_cnt_std',
       'bene_unique_cnt_min', 'bene_unique_cnt_max', 'bene_day_srvc_cnt_sum',
       'bene_day_srvc_cnt_mean', 'bene_day_srvc_cnt_median',
       'bene_day_srvc_cnt_std', 'bene_day_srvc_cnt_min',
       'bene_day_srvc_cnt_max', 'average_submitted_chrg_amt_sum',
       'average_submitted_chrg_amt_mean', 'average_submitted_chrg_amt_median',
       'average_submitted_chrg_amt_std', 'average_submitted_chrg_amt_min',
       'average_submitted_chrg_amt_max', 'average_medicare_payment_amt_sum',
       'average_medicare_payment_amt_mean',
       'average_medicare_payment_amt_median',
       'average_medicare_payment_amt_std', 'average_medicare_payment_amt_min',
       'average_medicare_payment_amt_max', 'EXCLTYPE',
       'EXCLDATE', 'REINDATE', 'WAIVERDATE', 'WVRSTATE',
       'MIN_EXCLUSION_PERIOD', 'END_EXCLDATE']]

In [37]:
partd = partd[['npi', 'specialty_description', 'DATA_YEAR', 'bene_count_sum', 'bene_count_mean', 'bene_count_median',
       'bene_count_std', 'bene_count_min', 'bene_count_max',
       'total_claim_count_sum', 'total_claim_count_mean',
       'total_claim_count_median', 'total_claim_count_std',
       'total_claim_count_min', 'total_claim_count_max',
       'total_30_day_fill_count_sum', 'total_30_day_fill_count_mean',
       'total_30_day_fill_count_median', 'total_30_day_fill_count_std',
       'total_30_day_fill_count_min', 'total_30_day_fill_count_max',
       'total_day_supply_sum', 'total_day_supply_mean',
       'total_day_supply_median', 'total_day_supply_std',
       'total_day_supply_min', 'total_day_supply_max', 'total_drug_cost_sum',
       'total_drug_cost_mean', 'total_drug_cost_median', 'total_drug_cost_std',
       'total_drug_cost_min', 'total_drug_cost_max', 'EXCLTYPE',
       'EXCLDATE', 'REINDATE', 'WAIVERDATE', 'WVRSTATE',
       'MIN_EXCLUSION_PERIOD', 'END_EXCLDATE']]

In [38]:
dmepos = dmepos[['npi','EXCLTYPE',
       'EXCLDATE', 'REINDATE', 'WAIVERDATE', 'WVRSTATE',
       'MIN_EXCLUSION_PERIOD', 'END_EXCLDATE', 'REFERRING_PROVIDER_TYPE',
       'REFERRING_PROVIDER_GENDER', 'DATA_YEAR', 'NUMBER_OF_SUPPLIERS_sum',
       'NUMBER_OF_SUPPLIERS_mean', 'NUMBER_OF_SUPPLIERS_median',
       'NUMBER_OF_SUPPLIERS_std', 'NUMBER_OF_SUPPLIERS_min',
       'NUMBER_OF_SUPPLIERS_max', 'NUMBER_OF_SUPPLIER_BENEFICIARIES_sum',
       'NUMBER_OF_SUPPLIER_BENEFICIARIES_mean',
       'NUMBER_OF_SUPPLIER_BENEFICIARIES_median',
       'NUMBER_OF_SUPPLIER_BENEFICIARIES_std',
       'NUMBER_OF_SUPPLIER_BENEFICIARIES_min',
       'NUMBER_OF_SUPPLIER_BENEFICIARIES_max', 'NUMBER_OF_SUPPLIER_CLAIMS_sum',
       'NUMBER_OF_SUPPLIER_CLAIMS_mean', 'NUMBER_OF_SUPPLIER_CLAIMS_median',
       'NUMBER_OF_SUPPLIER_CLAIMS_std', 'NUMBER_OF_SUPPLIER_CLAIMS_min',
       'NUMBER_OF_SUPPLIER_CLAIMS_max', 'NUMBER_OF_SUPPLIER_SERVICES_sum',
       'NUMBER_OF_SUPPLIER_SERVICES_mean',
       'NUMBER_OF_SUPPLIER_SERVICES_median', 'NUMBER_OF_SUPPLIER_SERVICES_std',
       'NUMBER_OF_SUPPLIER_SERVICES_min', 'NUMBER_OF_SUPPLIER_SERVICES_max',
       'AVG_SUPPLIER_MEDICARE_PMT_AMT_sum',
       'AVG_SUPPLIER_MEDICARE_PMT_AMT_mean',
       'AVG_SUPPLIER_MEDICARE_PMT_AMT_median',
       'AVG_SUPPLIER_MEDICARE_PMT_AMT_std',
       'AVG_SUPPLIER_MEDICARE_PMT_AMT_min',
       'AVG_SUPPLIER_MEDICARE_PMT_AMT_max',
       'AVG_SUPPLIER_SUBMITTED_CHARGE_sum',
       'AVG_SUPPLIER_SUBMITTED_CHARGE_mean',
       'AVG_SUPPLIER_SUBMITTED_CHARGE_median',
       'AVG_SUPPLIER_SUBMITTED_CHARGE_std',
       'AVG_SUPPLIER_SUBMITTED_CHARGE_min',
       'AVG_SUPPLIER_SUBMITTED_CHARGE_max']]

### Remove all rows without npi or provider info.

In [39]:
partb = partb[(partb.npi != 0) & (partb.provider_type)]
partd = partd[(partd.npi != 0) & (partd.specialty_description)]
dmepos = dmepos[(dmepos.npi != 0) & (dmepos.REFERRING_PROVIDER_TYPE)]

# Add Labels to each dataset

Add the target column

In [40]:
partb['TARGET'] = '0'
partd['TARGET'] = '0'
dmepos['TARGET'] = '0'

Extract the exclusion year into it's own column -- we will compare this to the data collection year then set the label as `FRAUD` or `NOT_FRAUD`.

In [41]:
partb['START_EXCLDATE'] = partb['EXCLDATE'].dt.year
partd['START_EXCLDATE'] = partd['EXCLDATE'].dt.year
dmepos['START_EXCLDATE'] = dmepos['EXCLDATE'].dt.year

In [42]:
partb

,npi,provider_type,DATA_YEAR,nppes_provider_gender,line_srvc_cnt_sum,line_srvc_cnt_mean,line_srvc_cnt_median,line_srvc_cnt_std,line_srvc_cnt_min,line_srvc_cnt_max,...,average_medicare_payment_amt_max,EXCLTYPE,EXCLDATE,REINDATE,WAIVERDATE,WVRSTATE,MIN_EXCLUSION_PERIOD,END_EXCLDATE,TARGET,START_EXCLDATE
60893,1124292966,Rheumatology,2016.0,M,1943.0,242.875000,109.0,250.453553,25.0,684.0,...,159.164286,1128a1,2020-06-18,NaT,NaT,NaN,5.0,2025.0,0,2020.0
60894,1124292966,Rheumatology,2017.0,M,1805.0,225.625000,167.0,197.418292,33.0,464.0,...,0.000000,1128a1,2020-06-18,NaT,NaT,NaN,5.0,2025.0,0,2020.0
60895,1124292966,Rheumatology,2018.0,M,1367.0,151.888889,74.0,164.602889,12.0,439.0,...,0.000000,1128a1,2020-06-18,NaT,NaT,NaN,5.0,2025.0,0,2020.0
60896,1679754725,Nephrology,2016.0,M,2844.0,203.142857,42.5,367.907358,15.0,1228.0,...,179.270000,1128b7,2020-04-22,NaT,NaT,NaN,NaN,NaN,0,2020.0
60897,1679754725,Nephrology,2017.0,M,3469.0,266.846154,43.0,490.889303,15.0,1674.0,...,0.000000,1128b7,2020-04-22,NaT,NaT,NaN,NaN,NaN,0,2020.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2976497,1992999825,Otolaryngology,2016.0,M,924.0,66.000000,40.0,69.859199,12.0,268.0,...,187.740000,NaN,NaT,NaT,NaT,NaN,NaN,NaN,0,NaN
2976498,1992999825,Otolaryngology,2017.0,M,923.0,65.928571,50.0,64.761651,12.0,248.0,...,0.000000,NaN,NaT,NaT,NaT,NaN,NaN,NaN,0,NaN
2976499,1992999825,Otolaryngology,2018.0,M,779.0,59.923077,51.0,55.562970,13.0,210.0,...,0.000000,NaN,NaT,NaT,NaT,NaN,NaN,NaN,0,NaN
2976500,1992999874,Internal Medicine,2016.0,F,984.0,140.571429,138.0,135.804832,13.0,323.0,...,174.520000,NaN,NaT,NaT,NaT,NaN,NaN,NaN,0,NaN


In [43]:
def make_labels(start_exc, data_yr):
    if start_exc > data_yr:
        return 'FRAUD'
    else:
        return 'NOT_FRAUD'
        

In [44]:
#applying the fraud labeling function to datasets

partb['TARGET'] = partb[["START_EXCLDATE","DATA_YEAR"]].apply(lambda x: make_labels(*x), axis=1)
partd['TARGET'] = partd[["START_EXCLDATE","DATA_YEAR"]].apply(lambda x: make_labels(*x), axis=1)
dmepos['TARGET'] = dmepos[["START_EXCLDATE","DATA_YEAR"]].apply(lambda x: make_labels(*x), axis=1)

As you can see from the outputs below, each of the datasets are highly imbalanced. This is an issue that would need to be revisited.

In [45]:
partd.TARGET.value_counts()

NOT_FRAUD    2763428
FRAUD           1007
Name: TARGET, dtype: int64

In [46]:
partb.TARGET.value_counts()

NOT_FRAUD    2910860
FRAUD            765
Name: TARGET, dtype: int64

In [47]:
dmepos.TARGET.value_counts()

NOT_FRAUD    883417
FRAUD           426
Name: TARGET, dtype: int64

# One Hot Encoding of Categorical Variables.

In [48]:
partb_category_columns = ['provider_type', 'nppes_provider_gender']
partd_category_columns = ['specialty_description']
dmepos_category_columns = ['REFERRING_PROVIDER_GENDER', 'REFERRING_PROVIDER_TYPE']

In [49]:
partb = pd.get_dummies(partb, columns=partb_category_columns, drop_first=True)
partd = pd.get_dummies(partd, columns=partd_category_columns, drop_first=True)
dmepos = pd.get_dummies(dmepos, columns=dmepos_category_columns, drop_first=True)

### Drop the columns that we don't need for modeling.

In [50]:
columns_to_drop = [ 'EXCLTYPE','EXCLDATE','REINDATE','WAIVERDATE',
                   'WVRSTATE','MIN_EXCLUSION_PERIOD','END_EXCLDATE',
                   'START_EXCLDATE', 'npi', 'DATA_YEAR']

partb.drop(columns_to_drop, axis=1, inplace=True)
partd.drop(columns_to_drop, axis=1, inplace=True)
dmepos.drop(columns_to_drop, axis=1, inplace=True)

In [51]:
partb.shape

(2911625, 126)

In [52]:
partd.shape

(2764435, 257)

In [53]:
dmepos.shape

(883843, 229)

### Fill in `NaN` values with `0`.

In [74]:
partd.fillna(0, inplace=True)
dmepos.fillna(0, inplace=True)

# Export data for modeling.

In [76]:
pd.to_pickle(partd, '/Volumes/ML_projects/Medicare_Fraud_Datasets/processed_data/partd.pkl')
pd.to_pickle(partb, '/Volumes/ML_projects/Medicare_Fraud_Datasets/processed_data/partb.pkl')
pd.to_pickle(dmepos, '/Volumes/ML_projects/Medicare_Fraud_Datasets/processed_data/dmepos.pkl')

In [81]:
partb.columns.to_list()

['line_srvc_cnt_sum',
 'line_srvc_cnt_mean',
 'line_srvc_cnt_median',
 'line_srvc_cnt_std',
 'line_srvc_cnt_min',
 'line_srvc_cnt_max',
 'bene_unique_cnt_sum',
 'bene_unique_cnt_mean',
 'bene_unique_cnt_median',
 'bene_unique_cnt_std',
 'bene_unique_cnt_min',
 'bene_unique_cnt_max',
 'bene_day_srvc_cnt_sum',
 'bene_day_srvc_cnt_mean',
 'bene_day_srvc_cnt_median',
 'bene_day_srvc_cnt_std',
 'bene_day_srvc_cnt_min',
 'bene_day_srvc_cnt_max',
 'average_submitted_chrg_amt_sum',
 'average_submitted_chrg_amt_mean',
 'average_submitted_chrg_amt_median',
 'average_submitted_chrg_amt_std',
 'average_submitted_chrg_amt_min',
 'average_submitted_chrg_amt_max',
 'average_medicare_payment_amt_sum',
 'average_medicare_payment_amt_mean',
 'average_medicare_payment_amt_median',
 'average_medicare_payment_amt_std',
 'average_medicare_payment_amt_min',
 'average_medicare_payment_amt_max',
 'TARGET',
 'provider_type_Advanced Heart Failure and Transplant Cardiology',
 'provider_type_All Other Suppliers',


In [80]:
partd.TARGET.value_counts()

NOT_FRAUD    2763428
FRAUD           1007
Name: TARGET, dtype: int64